# Find all must support dependent profiles in StructureDefinitions

prerequisite - need a completed local build

- see FHIR-39723
- for hard coded path search all SD for all reference with must support targets
  - including all references with a single target
  - exclude add'l USCDI
  - ignore choice of targets for now 
- associated url with title
- save as simple csv in IG data file
- use liquid template to geenerate table in IG

TODO ... for add'l USCDI most of these are choice of target

In [1]:
from pathlib import Path
from json import loads, dumps
import pandas as pd
snapshot_path =  Path('/Users/ehaas/Documents/FHIR/US-Core/output')  # use this  NEED a completed local build
my_path = snapshot_path
out_path = Path(r'/Users/ehaas/Documents/FHIR/US-Core/input/data')
ig_base_url = 'http://hl7.org/fhir/us/core/'  # US Core don't forget the trailing /
fhir_base = 'http://hl7.org/fhir/' # canonical url for FHIR don't forget the trailing /

addl_uscdi = False


### Update Profile Canonicals to Relative References 

In [2]:
def get_my_dict(addl_uscdi):
    title_map={}
    derived_map={}
    my_dict = {}
    excludelist = []
    # print(f'addl_uscdi={addl_uscdi}')
    whitelist = []

    count = 0
    cant_match_path = []

    for i in sorted(my_path.glob('Struct*.json')):
        # print(i, type(i))
        my_urls = set() # MS only set
        # my_uscdi_urls = set() # MS + USCDI set
        obj = loads(i.read_text()) #dict
        if obj['id'] in excludelist or obj['type'] == 'Extension':
            continue
        # type_map[obj['url']] = obj['type']
        # title_map[obj['url']] = obj['title']
        derived_map[obj['url']] = obj['baseDefinition']
        # print(enum,i,)
        # print(obj['url'])
        #find references type elements in diff
        for element in obj['differential']['element']:
            # print(element['id'])
            # print()
        
            try:
                targetProfile = (type for type in element['type'] if type['code'] == 'Reference').__next__()['targetProfile']
                # print(targetProfile)
            except KeyError:
                # find snapshot_element
                try:
                    snapshot_element =  (snapshot_element for snapshot_element in obj['snapshot']['element'] if snapshot_element['id'] == element['id']).__next__()
                    # check if snapshot_element has a type reference
                except StopIteration:
                    # print('id mismatch')
                    pass
                else:
                    try:
                        snapshot_reference = (type for type in snapshot_element['type'] if type['code'] == 'Reference').__next__()['targetProfile']
                        # print(f"++++++++++++++++snapshot_reference={snapshot_reference}+++++++++++++++++++++")
                        try:
                            if not addl_uscdi and element['mustSupport']: # only Must Supports / no add;l uscdi elements
                                my_urls.update(snapshot_reference) #add list to set
                        except KeyError: # Add;l USCDI
                            # my_uscdi_urls.update(snapshot_reference) #add list to set
                            pass

                    except KeyError:
                        # print('not a type = reference')
                        pass
                    except StopIteration:
                        # print('not a type = reference')
                        pass
            except StopIteration:
                # print('no reference')
                pass
            else:
                # print(f" for {obj['title']} {element['id']}:")
                # print(targetProfile)
                ms_ext = None
                try:
                    ms_ext = (type for type in element['type'] if type['code'] == 'Reference').__next__()['_targetProfile']
                except KeyError:
                    # print("no  _targetProfile")
                    pass
                except StopIteration:
                    # print("missing  _targetProfile")
                    pass

                # print(targetProfile, ms_ext)
                # print(f'ms_ext = {ms_ext}')
                # print(element['id'])
                # print(dict(zip(targetProfile,ms_ext)))
                # print(f" for {obj['title']} {element['id']} targetProfile = {targetProfile}")
                # print()
                try:
                    zipped = dict(zip(targetProfile,ms_ext))
                    ms_targetProfile = [k for k,v in zipped.items() if v["extension"][0]['valueBoolean']]
                    if ms_targetProfile:
                      try:
                          if not addl_uscdi and element['mustSupport']: # only Must Supports / no add;l uscdi elements
                              my_urls.update(ms_targetProfile) #add list to set
                          elif addl_uscdi and [e for e in element['extension'] if e['url'] == 'http://hl7.org/fhir/us/core/StructureDefinition/uscdi-requirement']:
                              my_urls.update(ms_targetProfile) #add list to set
                      except KeyError: # Must Supports + Add;l USCDI
                          # my_uscdi_urls.update(ms_targetProfile) #add list to set
                          pass
                    else:
                          # my_uscdi_urls.update(targetProfile)  #choice of targets
                          # print(f"Add'l USCDI choice of targetProfiles = {targetProfile}\n")
                          pass
                except TypeError:
                    print(f" for {obj['title']} {element['id']}:")
                    print(targetProfile)
                    try:
                        if not addl_uscdi and element['mustSupport'] and len(targetProfile) == 1 : # Must Supports with only 1 target/ not add;l uscdi elements
                            print(f"only 1 target in targetProfile = {targetProfile}\n")
                            my_urls.update(targetProfile) #add list to set
                        else: # element with > 1 choices not tagged with must support tag this is could be a profile error or a choice of at least one need.
                            print(f" for {obj['title']} {element['id']} >1 target in targetProfile = {targetProfile}\n")
                            pass
                    except KeyError: # Must Supports + Add;l USCDI
                        # print(f"Add'l USCID targetProfile = {targetProfile}\n")
                        # my_uscdi_urls.update(targetProfile) #add list to set
                        pass
        # print(f'my_uscdi_urls = {my_uscdi_urls}\n')
        # if addl_uscdi:
        #     my_dict[obj['url']] = list(my_uscdi_urls)
        # else:
        #     my_dict[obj['url']] = list(my_urls)
        # print(f'addl_uscdi={addl_uscdi}')
        # if addl_uscdi:
          #  my_dict[obj['url']] = [(i,True) for i in list(my_uscdi_urls)]
        else:
          my_dict[obj['url']] = list(my_urls)

    ### update profiles derived from other US Core profiles

    #- iterate through and combine lists of profiles from parent profiles.   
    for k,v in derived_map.items():
        if v in my_dict.keys():
            # print(k,v,my_dict[k],my_dict[v])
            my_dict[k] = my_dict[k] + my_dict[v]
    # print(f"my_dict={my_dict}")

    return my_dict

### Map the urls to titles and create CSV for use in IG as liquid template

In [3]:
ig_path = my_path / 'ImplementationGuide-hl7.fhir.us.core.json'
canonical = 'http://hl7.org/fhir/us/core/'
# print(type(ig_path))
# print(ig_path.read_text())
ig_obj = loads(ig_path.read_text()) #dict


def title_map(url):
  try:
    title = [resource['name'] for resource in ig_obj['definition']['resource'] if resource['reference']['reference'] == url.split(canonical)[-1]][0]
  except IndexError:
    #  print(f"no title for {url} use {url.split('/')[-1]}")
     title = url.split('/')[-1]
  return title 

def get_new_dict(my_dict): # map urls to Titles for rendering in IG
  new_dict = {}
  for (k,v) in my_dict.items():
      targets = [title_map(target) for target in v if target != 'http://hl7.org/fhir/StructureDefinition/Resource']  # remove generic resource target
      new_dict[title_map(k)] = sorted(targets)
  return new_dict
    

my_dict = get_my_dict(addl_uscdi)  #dictionary of references for each profile
print(f"my_dict={my_dict}")
new_dict = get_new_dict(my_dict)
print(f"new_dict = {new_dict}")

df = pd.DataFrame.from_dict(new_dict, orient='index' )  # transposes rows to columns and assigns the index to the profile row
df

 for US Core AllergyIntolerance Profile AllergyIntolerance.patient:
['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient']
only 1 target in targetProfile = ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient']

 for US Core Average Blood Pressure Profile Observation.subject:
['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient']
only 1 target in targetProfile = ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient']

 for US Core Care Experience Preference Profile Observation.subject:
['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient']
only 1 target in targetProfile = ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient']

 for US Core Condition Encounter Diagnosis Profile Condition.encounter:
['http://hl7.org/fhir/us/core/StructureDefinition/us-core-encounter']
only 1 target in targetProfile = ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-encounter']

 for US Core Coverage Profile Coverage.ben

,0,1,2,3,4,5,6
US Core Pediatric Head Occipital Frontal Circumference Percentile Profile,US Core Patient Profile,None,None,None,None,None,None
US Core Pediatric BMI for Age Observation Profile,US Core Patient Profile,None,None,None,None,None,None
US Core Pediatric Weight for Height Observation Profile,US Core Patient Profile,None,None,None,None,None,None
US Core ADI DocumentReference Profile,US Core Patient Profile,US Core Practitioner Profile,None,None,None,None,None
US Core AllergyIntolerance Profile,US Core Patient Profile,None,None,None,None,None,None
US Core Average Blood Pressure Profile,US Core Patient Profile,None,None,None,None,None,None
US Core Blood Pressure Profile,US Core Patient Profile,None,None,None,None,None,None
US Core BMI Profile,US Core Patient Profile,None,None,None,None,None,None
US Core Body Height Profile,US Core Patient Profile,None,None,None,None,None,None
US Core Body Temperature Profile,US Core Patient Profile,None,None,None,None,None,None


### Save as csv and update headers
  - this is a hack - save df as csv, retrieve csv and resave with headers.

In [4]:
file_path = out_path / 'ms_refs.csv'
df.to_csv(file_path)
df = pd.read_csv(file_path)
headers = ['profile','target1','target2','target3','target4','target5','target6','target7',]
df.to_csv(file_path, header=headers,)